In [1]:
!pip install cleanlab

     |████████████████████████████████| 77 kB 3.3 MB/s 


In [21]:
import gc
import os
import re
import string
from collections import Counter
import pandas as pd
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)
import numpy as np
from google.colab import drive
from nltk.stem import WordNetLemmatizer
import nltk
from sklearn.naive_bayes import MultinomialNB
from sklearn import metrics
from sklearn.feature_extraction.text import TfidfVectorizer
import cleanlab
from cleanlab.classification import LearningWithNoisyLabels
from tqdm import tqdm
import plotly.express as px
nltk.download('stopwords')
nltk.download('punkt')
nltk.download('wordnet')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [3]:
path_drive = '/content/drive'
path_data = os.path.join(path_drive, 'MyDrive','Colab\ Notebooks', 'kaggle', 'data')
drive.mount(path_drive)

Mounted at /content/drive


In [4]:
%cp {path_data} -r ./
!git clone https://github.com/jespinosal/nlp_tools.git
%cd nlp_tools/

Cloning into 'nlp_tools'...
remote: Enumerating objects: 71, done.
remote: Counting objects: 100% (71/71), done.
remote: Compressing objects: 100% (45/45), done.
remote: Total 71 (delta 37), reused 58 (delta 24), pack-reused 0
Unpacking objects: 100% (71/71), done.
/content/nlp_tools


In [48]:
from models.confident_learning import get_k_fold_class_probs, get_noisy_labels
from parser.preprocessing import clean_text, stopwords
from nltk.corpus import stopwords as stopwords_set
from utils.data_partition import set_cv_dataset_partitions
from models.confident_learning import get_k_fold_class_probs, get_noisy_labels

In [ ]:
TRAIN_DATA_PATH = "/content/data/jigsaw-toxic-comment-classification-challenge/train.csv"
VALID_DATA_PATH = "/content/data/jigsaw-toxic-severity-rating/validation_data.csv"

In [19]:
stop_words = set(stopwords_set.words("english")) 
lemmatizer = WordNetLemmatizer() 

df_train = pd.read_csv(TRAIN_DATA_PATH)
df_train = df_train.reset_index().rename(columns={'index':'comment_id'}) 
df_validation_data = pd.read_csv(VALID_DATA_PATH)

unrelevant_words = ['wiki','wikipedia','page']

df_train['comment_text'] = df_train['comment_text'].apply(lambda x: ''.join([w for w in clean_text(x) if w not in unrelevant_words]))
df_train['comment_text'] = df_train['comment_text'].apply(lambda x: ''.join([w for w in stopwords(x,stop_words)]))
df_train['comment_text'] = df_train['comment_text'].apply(lambda x: ''.join([w for w in lemmatizer.lemmatize(x)]))

df_validation_data['less_toxic'] = df_validation_data['less_toxic'].apply(lambda x: ''.join([w for w in clean_text(x) if w not in unrelevant_words]))
df_validation_data['less_toxic'] = df_validation_data['less_toxic'].apply(lambda x: ''.join([w for w in stopwords(x,stop_words)]))
df_validation_data['less_toxic'] = df_validation_data['less_toxic'].apply(lambda x: ''.join([w for w in lemmatizer.lemmatize(x)]))

df_validation_data['more_toxic'] = df_validation_data['more_toxic'].apply(lambda x: ''.join([w for w in clean_text(x) if w not in unrelevant_words]))
df_validation_data['more_toxic'] = df_validation_data['more_toxic'].apply(lambda x: ''.join([w for w in stopwords(x,stop_words)]))
df_validation_data['more_toxic'] = df_validation_data['more_toxic'].apply(lambda x: ''.join([w for w in lemmatizer.lemmatize(x)]))


# Create a score that messure how much toxic is a comment
cat_mtpl = {'obscene': 0.10, 'toxic': 0.20, 'threat': 0.5, 
            'insult': 0.62, 'severe_toxic': 0.7, 'identity_hate': 1}

for category in cat_mtpl:
    df_train[category] = df_train[category] * cat_mtpl[category]

df_train['score'] = df_train.loc[:, 'toxic':'identity_hate'].sum(axis=1)


#Balance the clases
n_samples_toxic = len(df_train[df_train['score'] != 0])
n_samples_normal = len(df_train) - n_samples_toxic

idx_to_drop = df_train[df_train['score'] == 0].index[n_samples_toxic//5:]
df_train = df_train.drop(idx_to_drop)

print(f'Reduced number of neutral text samples from {n_samples_normal} to {n_samples_toxic//5}.')
print(f'Total number of training samples: {len(df_train)}')

#Feature engineering for creating 
df_tragets = pd.DataFrame(pd.unique(df_train['score'].values), columns=['target_value']).sort_values(by='target_value', ascending = True).reset_index(drop=True)
THRESHOLD = df_tragets['target_value'].quantile(q=0.2)
df_train['sentiment'] = df_train['score'].map(lambda x: 0 if x < THRESHOLD else 1 if x < THRESHOLD*2 else 2 if x < THRESHOLD*3 else 3 if x < THRESHOLD*4 else 4)
df_train = df_train.loc[df_train.comment_text.str.len() >3,]
df_train = df_train[['comment_text','sentiment', 'comment_id']].reset_index(drop=True)

Reduced number of neutral text samples from 143346 to 3245.
Total number of training samples: 19470


## Baseline model training with noise

In [22]:
tf_idf_vect = TfidfVectorizer(analyzer='word',stop_words= 'english', max_features=10500) 
X = tf_idf_vect.fit_transform(df_train['comment_text'])
model = MultinomialNB()
model.fit(X, df_train['sentiment'])
# Validate
X_less_toxic = tf_idf_vect.transform(df_validation_data['less_toxic'])
X_more_toxic = tf_idf_vect.transform(df_validation_data['more_toxic'])
p1 = model.predict_proba(X_less_toxic)
p2 = model.predict_proba(X_more_toxic)
print(f"Accuracy validation dataset: {(p1[:, 1] < p2[:, 1]).mean()}") 

Accuracy validation dataset: 0.6330543377175502


## Baseline model confident learning default implementation

In [28]:
tf_idf_vect = TfidfVectorizer(analyzer='word',stop_words= 'english', max_features=5000, norm='l2') # Library fails when vector features are poor

X = tf_idf_vect.fit_transform(df_train['comment_text'])
base_model = MultinomialNB()
model = LearningWithNoisyLabels(clf=base_model,
                                seed=100,
                                cv_n_folds=10,
                                prune_method='both',
                                converge_latent_estimates=False,
                                pulearning=None,
                                n_jobs=2)
model.fit(X=X, s=df_train['sentiment'].values) # noise_mask, sample_weight 

# Validate
X_less_toxic = tf_idf_vect.transform(df_validation_data['less_toxic'])
X_more_toxic = tf_idf_vect.transform(df_validation_data['more_toxic'])
p1 = model.predict_proba(X_less_toxic)
p2 = model.predict_proba(X_more_toxic)
print(f"Accuracy validation dataset: {(p1[:, 1] < p2[:, 1]).mean()}")

Accuracy validation dataset: 0.6194366945662283


## Baseline model confident learning custom implementation

In [53]:
folds=10
feature_transformer = TfidfVectorizer(analyzer='word',stop_words= 'english', max_features=10500)
classifier = MultinomialNB
df_train_ = set_cv_dataset_partitions(df=df_train, stratify_column='sentiment', k_folds=folds, seed=100)
df_train_ = get_k_fold_class_probs(df=df_train_,transformation=feature_transformer, baseline_model=classifier, k_folds=10)
multi_label = False
pyx= np.array(df_train_['predictions_probs'].to_list()) #  #This is the probability distribution of probabilities for each of the N examples x. 
s = df_train['sentiment'].values # A discrete vector of labels, s, which may contain mislabeling. “s”
cj_only_bool_mask, label_errors_idx_sorted, label_errors_idx = get_noisy_labels(pyx=pyx, s=s, multi_label=multi_label, verbose=True)

Total found error indices 6779 like [ 6 15 27 42 55] ..
Total components on the matrix 19462
confident join marix [[8675 1171    9    1 1562]
 [2020 3088   34    0  653]
 [ 355 1369   39    0  123]
 [  47  257   17    0   11]
 [   1   29    0    0    1]]


In [46]:
print('IDs with errors filtered', len(label_errors_idx))
print('IDs with errors', len(label_errors_idx_sorted))

IDs with errors filtered 4778
IDs with errors 6779


In [54]:
# Training with noise
# Note: index_id is used such as noise dataset reference and commend_id such as 
# Optimize the threshold filtering the top_k noisy labels
top_k_max = len(label_errors_idx_sorted)
steps = 10
noise_top_k = list(range(1, top_k_max, steps))
accuracy_list = []
tf_idf_vect = TfidfVectorizer(analyzer='word',stop_words= 'english', max_features=10500)
#tf_idf_vect = TfidfVectorizer(analyzer='char_wb', ngram_range=(2, 3) )
tf_idf_vect.fit(df_train_['comment_text'])
X_less_toxic = tf_idf_vect.transform(df_validation_data['less_toxic'])
X_more_toxic = tf_idf_vect.transform(df_validation_data['more_toxic'])
for top_k in tqdm(noise_top_k):
    index_to_filter = df_train_.index_id.isin(label_errors_idx_sorted[0:top_k])
    commend_id_filter = df_train_.loc[index_to_filter,'comment_id']
    df_train_filtered = df_train_.loc[~df_train_.comment_id.isin(commend_id_filter),:]
    X = tf_idf_vect.transform(df_train_filtered['comment_text'])
    model = MultinomialNB()
    model.fit(X, df_train_filtered['sentiment'])
    # Validate
    p1 = model.predict_proba(X_less_toxic)
    p2 = model.predict_proba(X_more_toxic)
    accuracy_val = (p1[:, 1] < p2[:, 1]).mean()
    accuracy_list.append(accuracy_val)

print(f"Accuracy validation dataset mean: {np.mean(accuracy_list)}")
print(f"Accuracy validation dataset max: {np.max(accuracy_list)} for {noise_top_k[np.argmax(accuracy_list)]} threshold ")
fig = px.scatter(x=noise_top_k, y=accuracy_list, color=accuracy_list) # todo coloour the label
fig.show()

100%|██████████| 678/678 [05:32<00:00,  2.04it/s]


Accuracy validation dataset mean: 0.628541870701071
Accuracy validation dataset max: 0.634582170851601 for 101 threshold 


In [67]:
top_k = 101
index_to_filter = df_train_.index_id.isin(label_errors_idx_sorted[0:top_k])
commend_id_filter = df_train_.loc[index_to_filter,'comment_id']
df_train_filtered = df_train_.loc[~df_train_.comment_id.isin(commend_id_filter),:]
print(f"Error frequencies by label for the top_k {top_k}:", Counter(df_train_.loc[index_to_filter,'sentiment']))

Error frequencies by label for the top_k 101: Counter({1: 67, 2: 31, 3: 2, 4: 1})


In [73]:
samples = 10
print('\n')
index_to_filter_last = df_train_.index_id.isin(label_errors_idx_sorted[top_k-samples:top_k])
print(f"Loss margin error in top k {samples} samples")
print(df_train_.loc[index_to_filter_last, ['comment_text', 'sentiment', 'predictions_probs']])



Loss margin error in top k 10 samples
                                            comment_text  sentiment                                  predictions_probs
5026   fuck problem bitch fuck delete dreamtime festi...          2  [0.09691501863761377, 0.8601147391372819, 0.04...
6811   general suckitude get cock mouth ban cant beli...          2  [0.8258721323333437, 0.1639007225442004, 0.009...
7556   fuck bertie thank taking time make comments re...          1  [0.9065212952448544, 0.08969768834257354, 0.00...
9827   wtf place nazi land geezus everything peer rev...          2  [0.8369486423121059, 0.14063907801503797, 0.02...
9854   talking little boys like sure utter waste time...          1  [0.9069456908364637, 0.09109598793237296, 0.00...
11545  racial jew fool want argue appropriate discuss...          2  [0.8370522457042614, 0.13937722168413127, 0.02...
13536  text used proof want sue guys call free encycl...          1  [0.9073338094459872, 0.09147058342833084, 0.00...
14174  y